In [1]:
from enum import Enum

import instructor
import pandas as pd
from pydantic import Field

from cuery import Prompt, Response, Task, pprint
from cuery.cli import set_env_vars

set_env_vars(apify_secrets=False)

{'APIFY_TOKEN': 'apify_api_avBHdevvEnZQQNh03F65ThDKODJhqg0imvTZ',
 'GOOGLE_ADS_DEVELOPER_TOKEN': 'dugP_nV5LLe5bIqOQfRuCw',
 'GOOGLE_ADS_LOGIN_CUSTOMER_ID': '6560490700',
 'GOOGLE_ADS_USE_PROTO_PLUS': 'True',
 'GOOGLE_ADS_JSON_KEY': 'eyJ0eXBlIjogInNlcnZpY2VfYWNjb3VudCIsICJwcm9qZWN0X2lkIjogImdyYXBoZXh0LWRldmVsb3BtZW50IiwgInByaXZhdGVfa2V5X2lkIjogIjYyZDU5NTAyNTU1NDFkYjZlNTY0ZmIwZGJhZjYwNjM5YjIzYTQ5NGQiLCAicHJpdmF0ZV9rZXkiOiAiLS0tLS1CRUdJTiBQUklWQVRFIEtFWS0tLS0tXG5NSUlFdmdJQkFEQU5CZ2txaGtpRzl3MEJBUUVGQUFTQ0JLZ3dnZ1NrQWdFQUFvSUJBUUNwKy9kemZZbkltNVg5XG5GWERFVEZzQk45VEgwaXMxMFBaOHJpVkdNbVVINHlzMTY2RTNSdGl1OGFnditNYTRTbmdZMi9SNEVMME5HN2VrXG5pd3EwK3cyT2c1Rldhd05ucjE2cGM0blR6ZFN0Y2x0cjEzNDZYYkVOa0duaUFjVWtNaC9YbHVQZkNOWmd6L3RtXG52Q2xuc2QyZUlqVy9ZVEszNkJZbHpWdU0wQ204TkFITU9pSHJ0bWFPUDlzMUhweS96NzdHMGZwanRnVFVnZjJjXG5senh1Wko3dDFRNFh6TUpRdytLYnZYdUVEdlVvRm1jZUgvODBVZXpBSkg2d1NiNGxWd2xKZ0xVaHNoTzBnbXlWXG5oek0rOW4vaHJyVkdHeDJPd2I4eU9tUjEyOFNjQks4eDZ5OVJ0a2cxU3JzWWFDbEhNV0hQQjJKNUxJeVg0cFdqXG5PZUJvTU1

# Create a prompt from simple string
The `Prompt` class expects a list of (jinja) messages with their roles. But it can also be instantiated from a simple string.

In [2]:
t = "Hello {{name}}! {% for item in ingredients %} {{ item }} {% endfor %}"
p = Prompt.from_string(t)
pprint(p)

╭──────────────────────────────────────────────────── PROMPT ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Required: ['ingredients', 'name']                                                                              │
│                                                                                                                 │
│ ╭─────────────────────────────────────────────────── USER ────────────────────────────────────────────────────╮ │
│ │ Hello {{name}}! {% for item in ingredients %} {{ item }} {% endfor %}                                       │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Simplified client/model creation

In [3]:
# Docstring descriptions will be passed via the response model to the LLM
class Recipe(Response):
    ingredients: list[str]
    """A list of ingredients for the dish."""


prompt = Prompt.from_string("Generate a list of recipe ingredients to make '{{dish}}'.")
task = Task(prompt=prompt, response=Recipe)
pprint(task)

model = "openai/gpt-4.1-mini"  # or e.g. "perplexity/sonar"
responses = await task(context=[{"dish": "spaghetti carbonara"}], model=model)
# responses.to_pandas(explode=False)

╭───────────────────────────────────────────────────── TASK ──────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── PROMPT ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │  Required: ['dish']                                                                                         │ │
│ │                                                                                                             │ │
│ │ ╭───────────────────────────────────────────────── USER ──────────────────────────────────────────────────╮ │ │
│ │ │ Generate a list of recipe ingredients to make '{{dish}}'.                                               │ │ │
│ │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│ ╭────── RESPONSE: Recipe ──────╮                                                                                │
│ │                              │                                                                                │
│ │ ╭─ ingredients: list[str] ─╮ │                                                                                │
│ │ │                          │ │                                                                                │
│ │ │  {'required': True}      │ │                                                                                │
│ │ │                          │ │                                                                                │
│ │ ╰──────────────────────────╯ │                                                                                │
│ │                              │                                                                                │
│ ╰──────────────────────────────╯                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Iterating context:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
responses.to_pandas(explode=False)

,dish,ingredients
0,spaghetti carbonara,"[spaghetti, eggs, Pecorino Romano cheese, guan..."


# Inspect LLM queries (containing final prompt send to LLM)

Only available (for now), when multiple rows were processed!

Each task maintains a log of errors and the queries to the LLM provider. Note that the structure of what's sent to the provider may be different for each.

In [5]:
task.queries[0]

{'messages': [{'role': 'user',
   'content': "Generate a list of recipe ingredients to make 'spaghetti carbonara'."}],
 'model': 'gpt-4.1-mini',
 'tools': [{'type': 'function',
   'function': {'name': 'Recipe',
    'description': 'Correctly extracted `Recipe` with all the required parameters with correct types',
    'parameters': {'additionalProperties': False,
     'properties': {'ingredients': {'items': {'type': 'string'},
       'title': 'Ingredients',
       'type': 'array'}},
     'required': ['ingredients'],
     'type': 'object'}}}],
 'tool_choice': {'type': 'function', 'function': {'name': 'Recipe'}}}

# Choices (enum)
Require LLM to respond with one of N _options_ (fixed categories).

In [6]:
class Role(Enum):
    PRINCIPAL = "PRINCIPAL"
    TEACHER = "TEACHER"
    STUDENT = "STUDENT"
    OTHER = "OTHER"


class UserDetail(Response):
    age: int
    name: str
    role: Role = Field(description="Correctly assign one of the predefined roles to the user.")


prompt = Prompt.from_string("Please a create a synthetic user profile with age, name and role.")
task = Task(prompt=prompt, response=UserDetail)

response = await task(model="openai/gpt-3.5-turbo")
print(response)
response.to_pandas()

[UserDetail(age=30, name='John Doe', role=<Role.STUDENT: 'STUDENT'>)]


,age,name,role
0,30,John Doe,STUDENT


Or using the Literal type

In [7]:
from typing import Literal


class UserDetail(Response):
    age: int
    name: str
    role: Literal["PRINCIPAL", "TEACHER", "STUDENT", "OTHER"]
    """Correctly assign one of the predefined roles to the user."""


response = await Task(prompt=prompt, response=UserDetail)(model="openai/gpt-3.5-turbo")
response.to_pandas()


,age,name,role
0,25,Alice,STUDENT


# Simple Multivalued fields
Require LLM to respond with a _list_ of values (unconstrained).

In [8]:
class Ingredients(Response):
    items: list[str] = Field(description="List of ingredients for the recipe.")


prompt = Prompt.from_string("List the ingredients for the following dish: {{dish}}.")
context = [{"dish": "pasta bolognese"}, {"dish": "chocolate cake"}]

task = Task(prompt=prompt, response=Ingredients)
responses = await task(context=context)
print(responses)

Iterating context:   0%|          | 0/2 [00:00<?, ?it/s]

[Ingredients(items=['pasta', 'ground beef', 'onion', 'garlic', 'carrot', 'celery', 'tomato paste', 'crushed tomatoes', 'red wine', 'beef broth', 'salt', 'pepper', 'olive oil', 'parmesan cheese', 'fresh parsley']), Ingredients(items=['chocolate', 'flour', 'sugar', 'butter', 'eggs', 'cocoa powder', 'baking powder', 'vanilla extract', 'salt', 'milk'])]


In [9]:
# Maintain the original structure of the responses
responses.to_pandas(explode=False)

,dish,items
0,pasta bolognese,"[pasta, ground beef, onion, garlic, carrot, ce..."
1,chocolate cake,"[chocolate, flour, sugar, butter, eggs, cocoa ..."


In [10]:
# Explode the list of ingredients into separate rows
responses.to_pandas(explode=True)

,dish,items
0,pasta bolognese,pasta
1,pasta bolognese,ground beef
2,pasta bolognese,onion
3,pasta bolognese,garlic
4,pasta bolognese,carrot
5,pasta bolognese,celery
6,pasta bolognese,tomato paste
7,pasta bolognese,crushed tomatoes
8,pasta bolognese,red wine
9,pasta bolognese,beef broth


In [11]:
# Convert to simple python records
responses.to_records(explode=False)

[{'dish': 'pasta bolognese',
  'items': ['pasta',
   'ground beef',
   'onion',
   'garlic',
   'carrot',
   'celery',
   'tomato paste',
   'crushed tomatoes',
   'red wine',
   'beef broth',
   'salt',
   'pepper',
   'olive oil',
   'parmesan cheese',
   'fresh parsley']},
 {'dish': 'chocolate cake',
  'items': ['chocolate',
   'flour',
   'sugar',
   'butter',
   'eggs',
   'cocoa powder',
   'baking powder',
   'vanilla extract',
   'salt',
   'milk']}]

# Nested models
Define a more complicated output structure by referencing another response model. 

In this case a list of certain length containing instances of pre-defined response model.

In [14]:
class Sector(Response):
    sector: str = Field(
        description="Human-readable title(!) of the industrical sector (in NAICS taxonomy)",
        min_length=10,
        max_length=150,
    )
    subsector: str = Field(
        description="Human-readable title(!) of the industrial SUBsector (in NAICS taxonomy)",
        min_length=5,
        max_length=150,
    )
    sector_automation_potential: int = Field(
        description="A score from 1 to 10 indicating the sector's potential for automation",
        ge=0,
        le=10,
    )


class Sectors(Response):
    sectors: list[Sector] = Field(
        description="A list of 1 to 5 NAIC industrial sectors with their AI automation potential",
        min_length=1,
        max_length=5,
    )


sectors_prompt = Prompt.from_string(
    "List some industrial sector in the country of {{country}} that have great AI automation potential."
)

context = [{"country": "Germany"}, {"country": "United States"}, {"country": "Japan"}]
sectors_task = Task(prompt=sectors_prompt, response=Sectors)
responses = await sectors_task(context=context)

Iterating context:   0%|          | 0/3 [00:00<?, ?it/s]

[08/18/25 15:16:35] WARNING  Encountered: 1 response parsing errors!

In [15]:
responses.to_pandas(explode=True)

,country,sector,subsector,sector_automation_potential
0,Germany,Manufacturing,Automobile Manufacturing,8
1,Germany,Healthcare,Medical Technology,7
2,Germany,Finance and Insurance,Banking and Financial Services,9
3,Germany,Information and Communication,Software Development,8
4,United States,Information,"Data Processing, Hosting, and Related Services",8
5,United States,Manufacturing,Machinery Manufacturing,9
6,United States,Health Care and Social Assistance,Individual and Family Services,7
7,Japan,Manufacturing,Automobile Manufacturing,8
8,Japan,Healthcare,Medical Equipment Manufacturing,7
9,Japan,Retail Trade,E-commerce,9


# Chain tasks together
Run multiple tasks one after the other, collecting the results in a single DataFrame.

Keep in mind here that the names of inputs of one task must be the same as the names of outputs in the previous one.

Here we extract first some industrial sectors for each input country, and then some job roles within each sector.

In [16]:
# Re-uses "sectors" task from previous code cell (!)

from cuery import Chain


class Job(Response):
    job_role: str
    """Name of the job role (job title, less than 50 characters)"""
    job_description: str
    """A short description of the job role (less than 200 characters)"""
    job_automation_potential: int = Field(
        description="A score from 1 to 10 indicating the job's potential for automation",
        ge=0,
        le=10,
    )


class Jobs(Response):
    jobs: list[Job]
    """A list of jobs with their AI automation potential and reasons for that potential"""


jobs_prompt = Prompt.from_string(
    "List some job roles with great AI automation potential in the country of {{country}} and the sector '{{sector}}'"
)

context = pd.DataFrame(
    {
        "country": ["Germany", "United States", "Japan"],
        "PIB": [4.0, 5.0, 3.5],
    }
)

jobs_task = Task(prompt=jobs_prompt, response=Jobs)
chain = Chain(sectors_task, jobs_task)
responses = await chain(context=context)

[08/18/25 15:17:35] INFO     [1/2] Running task 'Sectors'

Iterating context:   0%|          | 0/3 [00:00<?, ?it/s]

[08/18/25 15:17:40] WARNING  Encountered: 2 response parsing errors!

                    INFO     [2/2] Running task 'Jobs'

Iterating context:   0%|          | 0/9 [00:00<?, ?it/s]

In [17]:
responses

,country,sector,subsector,sector_automation_potential,job_role,job_description,job_automation_potential
0,Germany,Manufacturing,Automotive Industry,8,Robotics Engineer,"Design, develop, and maintain robots for manuf...",9
1,Germany,Manufacturing,Automotive Industry,8,Automation Specialist,Implement and maintain automated systems in ma...,8
2,Germany,Manufacturing,Automotive Industry,8,Machine Learning Engineer,Develop algorithms for predictive maintenance ...,8
3,Germany,Manufacturing,Automotive Industry,8,Industrial Data Scientist,Use data analytics to optimize production proc...,7
4,Germany,Information,Software Development,9,Data Scientist,Analyzing and interpreting complex data to inf...,7
5,Germany,Information,Software Development,9,Machine Learning Engineer,Designing and implementing machine learning mo...,8
6,Germany,Information,Software Development,9,AI Consultant,Providing strategic guidance on AI implementat...,6
7,Germany,Information,Software Development,9,Big Data Analyst,"Analyzing large datasets, identifying trends a...",5
8,Germany,Health Care,Medical Technology,7,Medical Transcriptionist,Transcribing medical reports dictated by docto...,8
9,Germany,Health Care,Medical Technology,7,Radiologic Technologist,Performing diagnostic imaging examinations and...,6


# Tools

`Tools` are another thin level of abstraction to make `Tasks` configurable with a clear input interface (the output is already defined by a `Response` model). They're mostly useful with tasks (prompts and response models) that can be customized, i.e. which have configurable parameters that don't depend on the context of the data over which it will be iterated.

We use pydantic again to define the interface. This has the advantage that we can re-use a tool's interface directly for FastAPI endpoints, and therefore also directly as an MCP interface.

In [18]:
from typing import ClassVar

from cuery.cli import set_env_vars
from cuery.prompt import Prompt
from cuery.response import Response, ResponseClass
from cuery.tool import Tool

set_env_vars(apify_secrets=False)


class Jokes(Response):
    jokes: list[str]


class Joker(Tool):
    n_jokes: int
    topics: list[str]

    response_model: ClassVar[ResponseClass] = Jokes

    @property
    def prompt(self):
        # ${vars} will be substituted once initially. and so values will be constant when iterating over data
        # Jinja variables (and other Jinja syntax) will be evaluated for each request/row/context item
        instructions = "Create ${n_jokes} one-liners about {{topic}}."
        return Prompt(messages=instructions).substitute(n_jokes=self.n_jokes)

    @property
    def context(self):
        return [{"topic": topic} for topic in self.topics]


joker = Joker(n_jokes=3, topics=["cats", "nerds", "youths"])
result = await joker(n_concurrent=10)
result

Gathering responses: 100%|██████████| 3/3 [00:02<00:00,  1.12it/s]


[Jokes(jokes=['Why was the cat sitting on the computer? It wanted to keep an eye on the mouse!', 'What do you call a pile of cats? A meowtain!', "Why don't cats play poker in the jungle? Too many cheetahs!"]), Jokes(jokes=['Why do programmers prefer dark mode? Because light attracts bugs.', 'Why did the developer go broke? Because he used up all his cache.', 'Why was the math book sad? It had too many problems.']), Jokes(jokes=['Why did the student eat his homework? Because the teacher said it was a piece of cake!', "Why couldn't the bicycle stand up by itself? It was two tired!", "I asked the library if they had any books on paranoia. They whispered, 'They're right behind you…'"])]

In [19]:
result.to_records(explode=False)

[{'topic': 'cats',
  'jokes': ['Why was the cat sitting on the computer? It wanted to keep an eye on the mouse!',
   'What do you call a pile of cats? A meowtain!',
   "Why don't cats play poker in the jungle? Too many cheetahs!"]},
 {'topic': 'nerds',
  'jokes': ['Why do programmers prefer dark mode? Because light attracts bugs.',
   'Why did the developer go broke? Because he used up all his cache.',
   'Why was the math book sad? It had too many problems.']},
 {'topic': 'youths',
  'jokes': ['Why did the student eat his homework? Because the teacher said it was a piece of cake!',
   "Why couldn't the bicycle stand up by itself? It was two tired!",
   "I asked the library if they had any books on paranoia. They whispered, 'They're right behind you…'"]}]

In [20]:
joker.task.queries[0]

{'messages': [{'role': 'user', 'content': 'Create 3 one-liners about nerds.'}],
 'model': 'gpt-3.5-turbo',
 'tools': [{'type': 'function',
   'function': {'name': 'Jokes',
    'description': 'Correctly extracted `Jokes` with all the required parameters with correct types',
    'parameters': {'additionalProperties': False,
     'properties': {'jokes': {'items': {'type': 'string'},
       'title': 'Jokes',
       'type': 'array'}},
     'required': ['jokes'],
     'type': 'object'}}}],
 'tool_choice': {'type': 'function', 'function': {'name': 'Jokes'}}}

In [23]:
from cuery.cli import set_env_vars
from cuery.tools.flex import generic

set_env_vars(apify_secrets=False)

p = "I need a schema for users having a name and email"
t = generic.SchemaGenerator(instructions=p, model="openai/gpt-4.1")
r = await t()

In [25]:
from cuery import pprint

pprint(r.to_dict())

{
    'reasoning': "The user wants to extract user information consisting of a name and an email. Both fields are 
essential for identifying users and contacting them. The 'name' should be a required string with a description 
clarifying its purpose. The 'email' should also be required with the 'email' format specified to ensure validation 
of the email address. These fields are marked as required since both are fundamental user attributes.",
    'json_schema': {
        'type': 'object',
        'properties': {
            'name': {'type': 'string', 'description': 'The full name of the user.'},
            'email': {'type': 'string', 'format': 'email', 'description': "The user's email address."}
        },
        'required': ['name', 'email'],
        'additionalProperties': False
    }
}

# Web search

In [66]:
import instructor

from cuery import Field, Prompt, Response, Task, pprint


class Citation(Response):
    id: int
    url: str


class Place(Response):
    name: str = Field(..., description="Name of the restaurant.")
    address: str = Field(..., description="Address of the restaurant.")
    telephone: str = Field(..., description="Telephone number of the restaurant.")


class Places(Response):
    summary: str
    citations: list[Citation]


client = instructor.from_provider(
    "openai/gpt-4.1-mini",
    mode=instructor.Mode.RESPONSES_TOOLS_WITH_INBUILT_TOOLS,
    async_client=True,
)

response, completion = await client.responses.create_with_completion(
    input="What are some of the best places to eat Paella in Madrid, Spain? Return a list of restaurants with their name, address and telephone number.",
    tools=[
        {
            "type": "web_search_preview",
            "search_context_size": "low",
            "user_location": {
                "type": "approximate",
                "country": "ES",
                "city": "Madrid",
                "region": "Madrid",
            },
        }
    ],
    response_model=Places,
)


In [67]:
pprint(response.to_dict())

{
    'summary': 'Here are some recommended restaurants to eat Paella in Madrid, Spain, including their name, 
address, and telephone number.',
    'citations': [
        {'id': 1, 'url': 'http://casalucio.es?utm_source=openai'},
        {'id': 2, 'url': 'http://www.botin.es?utm_source=openai'},
        {'id': 3, 'url': 'http://www.labarraca.es?utm_source=openai'},
        {'id': 4, 'url': 'http://www.lacasavalencia.es?utm_source=openai'},
        {'id': 5, 'url': 'http://www.elcluballard.com?utm_source=openai'},
        {'id': 6, 'url': 'http://www.lapaelladelareina.com/?utm_source=openai'}
    ]
}

In [69]:
pprint(completion)

Response(
    id='resp_6847f6e9a26081a3be30dfa0397be1b605a3d309438af1d8',
    created_at=1749546729.0,
    error=None,
    incomplete_details=None,
    instructions=None,
    metadata={},
    model='gpt-4.1-mini-2025-04-14',
    object='response',
    output=[
        ResponseFunctionWebSearch(
            id='ws_6847f6f20a3081a3a87f96dc3c282fe105a3d309438af1d8',
            status='completed',
            type='web_search_call'
        ),
        ResponseOutputMessage(
            id='msg_6847f6f4df6481a3bb498e559d99644a05a3d309438af1d8',
            content=[
                ResponseOutputText(
                    annotations=[
                        AnnotationURLCitation(
                            end_index=131,
                            start_index=80,
                            title='Casa Lucio',
                            type='url_citation',
                            url='http://casalucio.es?utm_source=openai'
                        ),
                        AnnotationURLCitation(
                            end_index=366,
                            start_index=320,
                            title='Botín',
                            type='url_citation',
                            url='http://www.botin.es?utm_source=openai'
                        ),
                        AnnotationURLCitation(
                            end_index=675,
                            start_index=620,
                            title='La Barraca',
                            type='url_citation',
                            url='http://www.labarraca.es?utm_source=openai'
                        ),
                        AnnotationURLCitation(
                            end_index=945,
                            start_index=879,
                            title='Casa de Valencia',
                            type='url_citation',
                            url='http://www.lacasavalencia.es?utm_source=openai'
                        ),
                        AnnotationURLCitation(
                            end_index=1210,
                            start_index=1150,
                            title='Club Allard',
                            type='url_citation',
                            url='http://www.elcluballard.com?utm_source=openai'
                        ),
                        AnnotationURLCitation(
                            end_index=1486,
                            start_index=1410,
                            title='La Paella de la Reina',
                            type='url_citation',
                            url='http://www.lapaelladelareina.com/?utm_source=openai'
                        )
                    ],
                    text="Here are some of the best places to enjoy authentic paella in Madrid, Spain:\n\n**[Casa 
Lucio](http://casalucio.es?utm_source=openai)**  \n**Closed · Spanish · $$ · 4.3 (741 reviews)**  \n_C. Cava Baja, 
35, 28005 Madrid Comunidad de Madrid_  \nFamous for its traditional Spanish cuisine, including a renowned paella 
dish.\n\n**[Botín](http://www.botin.es?utm_source=openai)**  \n**Closed · Spanish · $$$ · 4.2 (1216 reviews)**  
\n_Calle de los Cuchilleros, 17, LOC;DUP 17-19;, 28005 Madrid Comunidad de Madrid_  \nEstablished in 1725, this 
historic restaurant offers a variety of Spanish dishes, including a delectable paella.\n\n**[La 
Barraca](http://www.labarraca.es?utm_source=openai)**  \n**Closed · Paella · $$$ · 3.7 (386 reviews)**  \n_C. de la
Reina, 29, 28004 Madrid Comunidad de Madrid_  \nSpecializes in traditional Valencian paella, offering a range of 
seafood and meat options.\n\n**[Casa de Valencia](http://www.lacasavalencia.es?utm_source=openai)**  \n**Closed · 
Paella · $$$ · 3.9 (122 reviews)**  \n_P. Pintor Rosales, 58, 28008 Madrid Comunidad de Madrid_  \nKnown for its 
authentic Valencian paella, prepared with fresh, high-quality ingredients.\n\n**[Club 
Allard](http://www.elcluballard.com?utm_source=openai)**  \n**Closed · Spanish · $$ · 4.2

In [70]:
pprint(response.citations)

[
    Citation(id=1, url='http://casalucio.es?utm_source=openai'),
    Citation(id=2, url='http://www.botin.es?utm_source=openai'),
    Citation(id=3, url='http://www.labarraca.es?utm_source=openai'),
    Citation(id=4, url='http://www.lacasavalencia.es?utm_source=openai'),
    Citation(id=5, url='http://www.elcluballard.com?utm_source=openai'),
    Citation(id=6, url='http://www.lapaelladelareina.com/?utm_source=openai')
]

In [73]:
response.summary

'Here are some recommended restaurants to eat Paella in Madrid, Spain, including their name, address, and telephone number.'

In [71]:
completion.output[1].content[0]

ResponseOutputText(annotations=[AnnotationURLCitation(end_index=131, start_index=80, title='Casa Lucio', type='url_citation', url='http://casalucio.es?utm_source=openai'), AnnotationURLCitation(end_index=366, start_index=320, title='Botín', type='url_citation', url='http://www.botin.es?utm_source=openai'), AnnotationURLCitation(end_index=675, start_index=620, title='La Barraca', type='url_citation', url='http://www.labarraca.es?utm_source=openai'), AnnotationURLCitation(end_index=945, start_index=879, title='Casa de Valencia', type='url_citation', url='http://www.lacasavalencia.es?utm_source=openai'), AnnotationURLCitation(end_index=1210, start_index=1150, title='Club Allard', type='url_citation', url='http://www.elcluballard.com?utm_source=openai'), AnnotationURLCitation(end_index=1486, start_index=1410, title='La Paella de la Reina', type='url_citation', url='http://www.lapaelladelareina.com/?utm_source=openai')], text="Here are some of the best places to enjoy authentic paella in Ma

In [72]:
idx = [(ann.start_index, ann.end_index) for ann in completion.output[1].content[0].annotations]
print(idx)
response.summary[idx[0][0] : idx[0][1]]


[(80, 131), (320, 366), (620, 675), (879, 945), (1150, 1210), (1410, 1486)]


'their name, address, and telephone number.'